---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [3]:
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime

def blight_model():
    def time(hearing_date, ticket_issued):
        if not hearing_date or type(hearing_date) != str:
            return 73
        hearing = datetime.strptime(hearing_date, '%Y-%m-%d %H:%M:%S')
        ticket = datetime.strptime(ticket_issued, '%Y-%m-%d %H:%M:%S')
        
        void = hearing - ticket
        return void.days
    training = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
    testing = pd.read_csv('test.csv')
    training = training[(training['compliance'] == 0) | (training['compliance'] ==1)]
    geolocation = pd.read_csv('latlons.csv')
    address = pd.read_csv('addresses.csv')
    address = address.set_index('address').join(geolocation.set_index('address'), how = 'left')
    training = training.set_index('ticket_id').join(address.set_index('ticket_id'))
    testing = testing.set_index('ticket_id'). join(address.set_index('ticket_id'))
    
    training = training[~training['hearing_date'].isnull()]
    training['time_gap'] = training.apply(lambda row: time(row['hearing_date'],
                                                    row['ticket_issued_date']),
                                   axis = 1)
    testing['time_gap'] = testing.apply(lambda row: time(row['hearing_date'],
                                                    row['ticket_issued_date']),
                                   axis = 1)
    feature_split = ['agency_name', 'state','disposition']
    training.lat.fillna(method = 'pad', inplace = True)
    training.lon.fillna(method='pad', inplace = True)
    training.state.fillna(method = 'pad', inplace =True)
    
    testing.state.fillna(method = 'pad', inplace = True)
    testing.lat.fillna(method = 'pad', inplace = True)
    testing.lon.fillna(method = 'pad', inplace = True)
    
    training = pd.get_dummies(training, columns = feature_split)
    testing = pd.get_dummies(testing, columns = feature_split)
    
    remove_train = ['payment_status','payment_date',
                   'balance_due','collection_status',
                   'compliance_detail']
    remove_both = ['fine_amount', 'violator_name', 'zip_code',
                   'country', 'city','inspector_name',
                   'violation_street_number',
                   'violation_street_name',
                   'violation_zip_code',
                   'violation_description',
                   'mailing_address_str_number', 
                   'mailing_address_str_name',
                   'non_us_str_code','ticket_issued_date',
                   'hearing_date', 'grafitti_status',
                   'violation_code']
    
    training.drop(remove_train, axis = 1, inplace = True)
    training.drop(remove_both, axis =1 , inplace = True)
    testing.drop(remove_both, axis = 1 , inplace = True)
    
    
    feature_train = training.columns.drop('compliance')
    feature_train_set = set(feature_train)
    
    for feature in set(feature_train):
        if feature not in testing:
            feature_train_set.remove(feature)
    feature_train = list(feature_train_set)
    
    x_train = training[feature_train]
    y_train = training.compliance
    x_test = testing[feature_train]
    
    scaler = MinMaxScaler()
    x_train_scale = scaler.fit_transform(x_train)
    x_test_scale = scaler.fit_transform(x_test)
    
    
    clf = MLPClassifier(hidden_layer_sizes = [100,20], alpha = 6,
                       random_state = 0, solver = 'lbfgs',
                       verbose = 0)
    clf.fit(x_train_scale, y_train)
    
    test_prob = clf.predict_proba(x_test_scale)[:,1]
    
    df_testing = pd.read_csv('test.csv', encoding = 'ISO-8859-1')
    df_testing['compliance'] = test_prob
    df_testing.set_index('ticket_id', inplace = True)

    
    
    return df_testing.compliance
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.011466
285362    0.006542
285361    0.012934
285338    0.020111
285346    0.021755
285345    0.020215
285347    0.024519
285342    0.281951
285530    0.014029
284989    0.022222
285344    0.021832
285343    0.010697
285340    0.011085
285341    0.024619
285349    0.042492
285348    0.040438
284991    0.030361
285532    0.020849
285406    0.010286
285001    0.012066
285006    0.008206
285405    0.014874
285337    0.008762
285496    0.015088
285497    0.013500
285378    0.014170
285589    0.025348
285585    0.012730
285501    0.014832
285581    0.006222
            ...   
376367    0.007060
376366    0.010345
376362    0.068591
376363    0.071397
376365    0.007060
376364    0.010345
376228    0.012860
376265    0.010986
376286    0.073493
376320    0.012050
376314    0.010654
376327    0.112778
376385    0.076105
376435    0.176996
376370    0.139417
376434    0.020743
376459    0.028396
376478    0.000877
376473    0.011192
376484    0.012679
376482    0.013069
37

In [4]:
predict = blight_model()
predict = predict.to_frame().reset_index()
predict

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,ticket_id,compliance
0,284932,0.011466
1,285362,0.006542
2,285361,0.012934
3,285338,0.020111
4,285346,0.021755
5,285345,0.020215
6,285347,0.024519
7,285342,0.281951
8,285530,0.014029
9,284989,0.022222


In [5]:
predict.shape

(61001, 2)